 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [736]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
from enum import Enum
#import scipy.sparse as sp
#from sklearn.datasets import load_iris


##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number



Pandas version: 1.4.3
Numpy version: 1.23.1


In [737]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [738]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [739]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False #--> GUI with active modes and phases, start condition [active modes list ] as bool (true false). Generarte excel which generete flow chart.


In [740]:
if import_mode: 
    ##Change working directory to temp/epic
    os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


Current Working Directory is:  C:\temp\epic


In [741]:
if import_mode:
    print(os.listdir(cwd)) #List directory

['AutomationCode.xlsx', 'Info.txt', 'OLD_PID.csv', 'PID - Piping and Instrument Diagram_406699_A01.vsdm', 'PID.xlsx', 'TND.xlsx', '~$$PID - Piping and Instrument Diagram_406699_A01.~vsdm', '~$AutomationCode.xlsx', '~$TND.xlsx']


#Random_mode:
Generate aleatories PID numbers and symbols.



In [742]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [743]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

Series and DataFrames

In [744]:
#pid_series = pd.Series(random_PID_symbols, random_PID_numbers)
#pid_series = pid_series.sort_index(axis=0,ascending=True)
#pid_series

#Import Tag number definitions as a pandas series
#tnd = pd.read_csv('TND.csv',names=['PID','Description'])
#tnd = pd.read_excel('TND.xlsx',)
#tnd

In [745]:
#PID_series = tnd['PID'].squeeze()
#pid_exist = PID_series.str.contains('.100')

In [746]:
#tnd.at[0,'Description']

In [747]:
#tnd_pid.at[1.100,'Description'] Works if index_col='PID'

Reading Excel File

We can read an excel file into a data frame:

pd.read_excel('myExcel.xlsx', index_col=['ColumnA'])

We can also load one sheet into a data frame:

pd.read_excel(open('myExcel.xlsx', 'rb'), sheet_name='Sheet1')

AUTOMATION CODE

In [748]:
#Load PID data
#df_data_excel = pd.read_excel('PID.xlsx', index_col='Prop_PID_Number')
df_data_excel = pd.read_excel('PID.xlsx')
df_data_excel = df_data_excel.sort_values('Prop_PID_Element') #Sorting by Element we get elements ordered and ready to compare
print(df_data_excel)

     ShapeKey Prop_PID_Element  Prop_PID_Number
274      1762                B            1.852
244     14496                C            1.155
245     14444                D            1.680
226     16120               EC            1.680
58      18405                F            1.759
..        ...              ...              ...
55      18416               YC            1.757
207     16178               YC            1.825
100     17727               YC            1.737
211     16173               YC            1.707
208     16176               YC            1.717

[303 rows x 3 columns]


In [749]:
#Remove ShapeKey from the dataframe
del df_data_excel['ShapeKey']

In [750]:
# dropping duplicate values
df_data_excel.drop_duplicates(inplace=True)

In [751]:
df_data_excel.count()

Prop_PID_Element    295
Prop_PID_Number     295
dtype: int64

In [752]:
#Drop elements like filters, catalizators, etc
for items in df_data_excel.index:

    if df_data_excel.at[items, 'Prop_PID_Element'] == 'B':
        print("Deleted a catalizator in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'F':
        print("Deleted a filter in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'C':
        print("Deleted a catalizator in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)

    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'D':
        print("Deleted a D element: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    
    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'S':
        print("Deleted a Overpressure valve in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)

    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'W':
        print("Deleted a W element: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    




Deleted a catalizator in:  274
Deleted a catalizator in:  244
Deleted a D element:  245
Deleted a filter in:  58
Deleted a filter in:  26
Deleted a filter in:  131
Deleted a filter in:  108
Deleted a filter in:  103
Deleted a filter in:  0
Deleted a filter in:  75
Deleted a filter in:  74
Deleted a filter in:  44
Deleted a filter in:  45
Deleted a filter in:  63
Deleted a filter in:  98
Deleted a filter in:  229
Deleted a filter in:  25
Deleted a filter in:  230
Deleted a filter in:  11
Deleted a filter in:  234
Deleted a filter in:  271
Deleted a filter in:  237
Deleted a filter in:  232
Deleted a filter in:  231
Deleted a filter in:  258
Deleted a filter in:  275
Deleted a filter in:  280
Deleted a filter in:  1
Deleted a filter in:  272
Deleted a Overpressure valve in:  252
Deleted a Overpressure valve in:  285
Deleted a W element:  12


In [753]:
df_data_excel.count()


Prop_PID_Element    263
Prop_PID_Number     263
dtype: int64

In [754]:
#Load Automation Code excel file
df_automation_code = pd.read_excel('AutomationCode.xlsx')
df_automation_code

,Prop_PID_Element,Automation_Code
0,B,* Buffer *
1,C,* Catalizator *
2,D,* D *
3,EC,Voltage Control
4,F,* Filter *
5,FA-,Flow Alarm -
6,FIA+,Flow Indication Alarm +
7,FICA±,Flow Indication Control Alarm ±
8,FSA-,Flow Switching Alarm -
9,GIA,Position Indication Alarm


In [755]:
#Load Tag Name Definition
tnd_pid = pd.read_excel('TND.xlsx')
#tnd_pid = pd.read_excel('TND.xlsx',index_col='PID')
tnd_pid

,PID,Description,Alarm (any),HW(Bool),HW(Analog),HW(Bool).1,HW(Analog).1
0,1.100,Intake air fan,Alarm_Nr: 1100.0,%E1100.0,%EW100.0,%A1100.0,%AW100.0
1,1.101,Filter 1,NaN,NaN,NaN,NaN,NaN
2,1.102,Filter 2,NaN,NaN,NaN,NaN,NaN
3,1.103,Filter 3,NaN,NaN,NaN,NaN,NaN
4,1.104,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
895,1.995,NaN,NaN,NaN,NaN,NaN,NaN
896,1.996,NaN,NaN,NaN,NaN,NaN,NaN
897,1.997,NaN,NaN,NaN,NaN,NaN,NaN
898,1.998,NaN,NaN,NaN,NaN,NaN,NaN


In [756]:
# Def of Lookup as in Microsoft Excel.
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [757]:
df_data_excel.count()

Prop_PID_Element    263
Prop_PID_Number     263
dtype: int64

In [758]:
print(df_data_excel.to_string())

    Prop_PID_Element  Prop_PID_Number
226               EC            1.680
89               FA-            1.330
70               FA-            2.300
122              FA-            1.320
33               FA-            1.310
177              FA-            1.300
206             FIA+            1.825
96             FICA±            1.748
106            FICA±            1.728
209            FICA±            1.718
210            FICA±            1.708
101            FICA±            1.738
56             FICA±            1.758
29              FSA-            1.338
83              FSA-            1.318
80              FSA-            1.328
197             FSA-            1.308
73              FSA-            2.308
193              GIA            1.550
195              GIA            1.555
16              GIA+            1.410
301             GIA+            1.400
302             GIA-            1.410
20              GIA-            1.400
93              GSA+            1.335
126         

In [759]:
#Add Automation_Code information
#df_data_excel['Automation_Code'] = df_data_excel['Prop_PID_Element'].apply(xlookup, args = (df_automation_code['Prop_PID_Element'], df_automation_code['Automation_Code']));

In [760]:
#print(df_data_excel.to_string())


In [761]:
#Group by PID number
element_list = df_data_excel.groupby(['Prop_PID_Number']).agg(list).copy()

In [762]:
type(element_list)

pandas.core.frame.DataFrame

In [763]:
element_list.count()

Prop_PID_Element    143
dtype: int64

In [764]:
print(element_list.to_string())

                        Prop_PID_Element
Prop_PID_Number                         
1.100                     [M, SC, V, XA]
1.108                              [PDI]
1.150                   [M, SC, TSA+, V]
1.158                              [PDI]
1.160                     [M, SC, V, XA]
1.200                     [M, SC, V, XA]
1.208                              [PDI]
1.210                     [M, SC, V, XA]
1.218                              [PDI]
1.240                     [M, SC, V, XA]
1.248                              [PDI]
1.300                    [FA-, QIA+, XA]
1.304                             [M, P]
1.305                 [GSA+, GSA-, V, Y]
1.308                             [FSA-]
1.310                    [FA-, QIA+, XA]
1.314                             [M, P]
1.315                 [GSA+, GSA-, V, Y]
1.318                             [FSA-]
1.320                    [FA-, QIA+, XA]
1.324                             [M, P]
1.325                 [GSA+, GSA-, V, Y]
1.328           

In [765]:
#Drop compound elements which are not part of the software list
#Drop elements like filters, catalizators, etc
for items in element_list.index:
    if element_list.at[items, 'Prop_PID_Element'] == ['PI', 'V']:
        print("Deleted a Pressure indicator in: ",items)
        element_list.drop(items,axis=0,inplace=True)

Deleted a Pressure indicator in:  1.855


In [766]:
element_list.count()

Prop_PID_Element    142
dtype: int64

In [767]:
#Deletion of manual valves, as we have no other information in the PID
element_list.drop(1.654,axis=0,inplace=True)
element_list.drop(1.656,axis=0,inplace=True)
element_list.drop(1.850,axis=0,inplace=True)
element_list.drop(1.863,axis=0,inplace=True)

In [768]:
element_list.count()

Prop_PID_Element    138
dtype: int64

In [769]:
##Add tag number definition - reset index first, then use xlookup function
element_list = element_list.reset_index()
element_list['Tag_Number_Definition'] = element_list['Prop_PID_Number'].apply(xlookup, args = (tnd_pid['PID'], tnd_pid['Description']));

In [770]:
print(element_list.to_string())

     Prop_PID_Number         Prop_PID_Element                 Tag_Number_Definition
0              1.100           [M, SC, V, XA]                        Intake air fan
1              1.108                    [PDI]           Diferential pressure sensor
2              1.150         [M, SC, TSA+, V]                       Exhaust air fan
3              1.158                    [PDI]           Diferential pressure sensor
4              1.160           [M, SC, V, XA]                       Exhaust air fan
5              1.200           [M, SC, V, XA]                     Recirculation fan
6              1.208                    [PDI]          Diferential pressure sensor 
7              1.210           [M, SC, V, XA]                     Recirculation fan
8              1.218                    [PDI]           Diferential pressure sensor
9              1.240           [M, SC, V, XA]                     Recirculation fan
10             1.248                    [PDI]           Diferential pressure

In [771]:
#print(element_list.loc[element_list['Prop_PID_Number'] == 1.100]['Prop_PID_Element'].values)

In [772]:
value =list( element_list.at[0, 'Prop_PID_Element'])

In [773]:
value

['M', 'SC', 'V', 'XA']

In [774]:
#Software Module Definition
SOFTWARE_MODE_DEFINITION = {
    "S_AAS_MAS_TCP" : "S_AAS_MAS_TCP",
    "S_AAS_MODBUS_TCP" : "S_AAS_MODBUS_TCP",
    "S_AAS_SERIAL" : "S_AAS_SERIAL",
    "S_ALARM" : "S_ALARM",
    "S_DRIVE" : "S_DRIVE",
    "S_ELMT_ON_OFF" : "S_ELMT_ON_OFF",
    "S_EMITTER_CMT" : "S_EMITTER_CMT",
    "S_FIPA" : "S_FIPA",
    "S_FLOW_METER" : "S_FLOW_METER",
    "S_GASKET" : "S_GASKET",
    "S_LEAK" : "S_LEAK",
    "S_LIMIT_HI_LO" : "S_LIMIT_HI_LO",
    "S_LIMIT_ON_OFF" : "S_LIMIT_ON_OFF",
    "S_LIMIT_SWITCH" : "S_LIMIT_SWITCH",
    "S_LINE" : "S_LINE",
    "S_MOTOR1" : "S_MOTOR1",
    "S_PLANT" : "S_PLANT",
    "S_PMS_MODBUS_ASCII" : "S_PMS_MODBUS_ASCII",
    "S_PMS_MODBUS_TCP" : "S_PMS_MODBUS_TCP",
    "S_PUMP_TEST" : "S_PUMP_TEST",
    "S_REG_CONT" : "S_REG_CONT",
    #S_REG = "S_REG"
    "S_SIS" : "S_SIS",
    "S_SKANFOG" : "SKANFOG",
    #S_TIMESYNCH = "S_TIMESYNCH"
    "S_VALVE_FLAP" : "S_VALVE_FLAP",
    #S_WEIGHT_SCALE_NG = "S_WEIGHT_SCALE_NG"
    #S_WEIGHT_SCALE = "S_WEIGHT_SCALE"
    "S_WEIGHT_SCALE2_NG" : "S_WEIGHT_SCALE2_NG",
    "S_LEADER_FOLLOWER" : "S_LEADER_FOLLOWER"
    }

In [775]:
element_list_sw_modules = ['SW_Module','SW_Module_Nr','Connection1','Connection1_Nr','Connection2','Connection2_Nr', 'Connection3','Connection3_Nr','Connection4','Connection4_Nr']
element_list[element_list_sw_modules] = ''
element_list

,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
0,1.100,"[M, SC, V, XA]",Intake air fan,,,,,,,,,,
1,1.108,[PDI],Diferential pressure sensor,,,,,,,,,,
2,1.150,"[M, SC, TSA+, V]",Exhaust air fan,,,,,,,,,,
3,1.158,[PDI],Diferential pressure sensor,,,,,,,,,,
4,1.160,"[M, SC, V, XA]",Exhaust air fan,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2.300,"[FA-, QIA+, XA]","""2.3"" not found!",,,,,,,,,,
134,2.304,"[M, P]","""2.304"" not found!",,,,,,,,,,
135,2.308,[FSA-],"""2.308"" not found!",,,,,,,,,,
136,2.350,"[P, XA]","""2.35"" not found!",,,,,,,,,,


In [776]:
#Set Pid numbers as column
#create a new dataframe with PID as index
element_list = element_list.set_index('Prop_PID_Number')
element_list


,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
Prop_PID_Number,,,,,,,,,,,,
1.100,"[M, SC, V, XA]",Intake air fan,,,,,,,,,,
1.108,[PDI],Diferential pressure sensor,,,,,,,,,,
1.150,"[M, SC, TSA+, V]",Exhaust air fan,,,,,,,,,,
1.158,[PDI],Diferential pressure sensor,,,,,,,,,,
1.160,"[M, SC, V, XA]",Exhaust air fan,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
2.300,"[FA-, QIA+, XA]","""2.3"" not found!",,,,,,,,,,
2.304,"[M, P]","""2.304"" not found!",,,,,,,,,,
2.308,[FSA-],"""2.308"" not found!",,,,,,,,,,


In [777]:
drive_number = 0
motor_number = 0
reg_number = 0
limit_hi_lo = 0
elmt_on_off = 0
valve_flap = 0
limit_switch = 0
sis = 0
weight_scale2_ng = 0
leak = 0
particle_counter = 0
s_aas_mas_tcp = 0
leader_follower = 0


for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

        #Drive
        case ['M', 'SC', 'TSA+', 'V']:
            #Drive
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_DRIVE']
            drive_number = drive_number + 1 
            element_list.at[items, 'SW_Module_Nr'] = drive_number

            #Motor Conn 1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            motor_number = motor_number + 1
            element_list.at[items, 'Connection1_Nr'] = motor_number

            #Reg Conn 2
            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            reg_number = reg_number + 1
            element_list.at[items, 'Connection2_Nr'] = reg_number

            #Limit hi lo conn3
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection3_Nr'] = (element_list.at[1.840, 'SW_Module_Nr'])

        #Motor
        case ['M', 'SC', 'V', 'XA'] | ['V', 'YC'] | ['H', 'M', 'NC'] | ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC'] | ['EC', 'TSA+'] | ['M', 'P', 'SC']: 
            #Motor + #Reg
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            motor_number = motor_number + 1
            element_list.at[items, 'SW_Module_Nr'] = motor_number

            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            reg_number = reg_number + 1
            element_list.at[items, 'Connection1_Nr'] = reg_number

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection2_Nr'] = -1

        #Limit hi lo
        case ['PDI'] | ['PDIS'] | ['TIC'] | ['FICA±'] | ['TICA±'] | ['MIA±'] | ['SIA±'] | ['M', 'P', 'QIA+', 'XA'] | ['QIA-'] | ['QSA+'] | ['PDICA±']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        #elmt_on_off
        case ['FA-', 'QIA+', 'QIA+', 'XA'] | ['M', 'P'] | ['L']:  
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_ELMT_ON_OFF']
            elmt_on_off = elmt_on_off + 1
            element_list.at[items, 'SW_Module_Nr'] = elmt_on_off

        #valve_flap
        case ['GSA+', 'GSA-', 'V', 'Y'] | ['GSA+', 'GSA-', 'K', 'Y'] | ['V']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_VALVE_FLAP']
            valve_flap = valve_flap + 1
            element_list.at[items, 'SW_Module_Nr'] = valve_flap

        #limit_switch
        case ['FSA-'] | ['PSA-', 'V'] | ['GZA-'] | ['GSA-'] | ['LSA-']: 
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_SWITCH']
            limit_switch = limit_switch + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_switch

        #WICA
        case ['WICA±']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_WEIGHT_SCALE2_NG']
            weight_scale2_ng = weight_scale2_ng + 1
            element_list.at[items, 'SW_Module_Nr'] = weight_scale2_ng

            #Limit hi lo conn1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = 'Any limit_hi_lo needed'

        #leak
        case ['FIA+', 'H', 'YC']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LEAK']
            leak = leak + 1
            element_list.at[items, 'SW_Module_Nr'] = leak

        #Servo mousehole
        case ['GIA+', 'GIA-', 'M', 'SC', 'XA']:
            #Limit hi lo conn1
            element_list.at[items, 'SW_Module'] = "S_SERVO_MOUSEHOLE"
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = 'tbd'
        
        #Particle counter
        case ['FA-', 'QIA+', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_PMS_MODBUS_TCP']
            particle_counter = particle_counter + 1
            element_list.at[items, 'SW_Module_Nr'] = particle_counter

        #Air Sampler MAS TCP
        case ['P', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_AAS_MAS_TCP']
            s_aas_mas_tcp = s_aas_mas_tcp + 1
            element_list.at[items, 'SW_Module_Nr'] = s_aas_mas_tcp

        #Airdryer
        case ['T', 'XA']:
            element_list.at[items, 'SW_Module'] = 'Air dryer Munter or similar'
            element_list.at[items, 'SW_Module_Nr'] = 1
            
        #case _:
            #print("Yikers. I don't know that one.")

    #print(items)

In [778]:
print(element_list.to_string())

                        Prop_PID_Element                 Tag_Number_Definition                    SW_Module SW_Module_Nr    Connection1          Connection1_Nr    Connection2 Connection2_Nr    Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                                                                       
1.100                     [M, SC, V, XA]                        Intake air fan                     S_MOTOR1            1     S_REG_CONT                       1  S_LIMIT_HI_LO             -1                                                         
1.108                              [PDI]           Diferential pressure sensor                S_LIMIT_HI_LO            1                                                                                                                              
1.150       

In [779]:
value =(element_list.at[1.840, 'SW_Module_Nr'])
print(value)

24


In [780]:
#Update with the S_limit hi lo value
for items in element_list.index:
    
    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
        #Drive
        case ['M', 'SC', 'TSA+', 'V']:

            #Limit hi lo conn3
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection3_Nr'] = (element_list.at[1.840, 'SW_Module_Nr'])

        #SIS Pumps
        case ['M', 'P', 'SC']:
            #Limit hi lo conn1
            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection2_Nr'] = (element_list.at[1.840, 'SW_Module_Nr']) #1.840
        
        #1.655 [H, M, NC] Control valve (Cool water control)
        case ['H', 'M', 'NC']: 
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection2_Nr'] = (element_list.at[1.688, 'SW_Module_Nr']) #1.688

        #1.680 [D, EC, TSA+]  Electric heating
        case ['EC', 'TSA+']: 
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection2_Nr'] = (element_list.at[1.688, 'SW_Module_Nr']) #1.688

        
         #Leader Follower
        case ['W']: #1.650 heat exchanger
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LEADER_FOLLOWER']
            leader_follower = leader_follower + 1
            element_list.at[items, 'SW_Module_Nr'] = leader_follower

            #Reg
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            reg_number = reg_number + 1
            element_list.at[items, 'Connection1_Nr'] = reg_number

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection2_Nr'] = (element_list.at[1.800, 'SW_Module_Nr'])#Temperature working chamber

            #Reg Conn 2
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            element_list.at[items, 'Connection3_Nr'] = (element_list.at[1.655, 'Connection1_Nr']) #Reg number of 1.655

            #Reg Conn 3
            element_list.at[items, 'Connection4'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            element_list.at[items, 'Connection4_Nr'] = (element_list.at[1.680, 'Connection1_Nr']) #Reg number of 1.680

        


In [817]:
if (1.655 and 1.688) in element_list.index:
   



AttributeError: 'DataFrame' object has no attribute 'concat'

In [782]:
print(element_list.to_string())

                        Prop_PID_Element                 Tag_Number_Definition                    SW_Module SW_Module_Nr    Connection1          Connection1_Nr    Connection2 Connection2_Nr    Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                                                                       
1.100                     [M, SC, V, XA]                        Intake air fan                     S_MOTOR1            1     S_REG_CONT                       1  S_LIMIT_HI_LO             -1                                                         
1.108                              [PDI]           Diferential pressure sensor                S_LIMIT_HI_LO            1                                                                                                                              
1.150       

In [783]:
#Drives
element_list_drives = element_list[element_list['SW_Module'] == 'S_DRIVE']
print(element_list_drives.to_string())

                 Prop_PID_Element Tag_Number_Definition SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr    Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                              
1.15             [M, SC, TSA+, V]       Exhaust air fan   S_DRIVE            1    S_MOTOR1              2  S_REG_CONT              2  S_LIMIT_HI_LO             24                           


In [784]:
#Motor
element_list_motors = element_list[element_list['SW_Module'] == 'S_MOTOR1']
print(element_list_motors.to_string())

                Prop_PID_Element        Tag_Number_Definition SW_Module SW_Module_Nr Connection1 Connection1_Nr    Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                    
1.100             [M, SC, V, XA]               Intake air fan  S_MOTOR1            1  S_REG_CONT              1  S_LIMIT_HI_LO             -1                                                      
1.160             [M, SC, V, XA]              Exhaust air fan  S_MOTOR1            3  S_REG_CONT              3  S_LIMIT_HI_LO             -1                                                      
1.200             [M, SC, V, XA]            Recirculation fan  S_MOTOR1            4  S_REG_CONT              4  S_LIMIT_HI_LO             -1                                                      
1.210             [M

In [785]:
#Valve
element_list_valve_flap = element_list[element_list['SW_Module'] == 'S_VALVE_FLAP']
print(element_list_valve_flap.to_string())

                   Prop_PID_Element                 Tag_Number_Definition     SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                                 
1.305            [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            1                                                                                                            
1.315            [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            2                                                                                                            
1.325            [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            3                                                                

In [786]:
#Element on off
element_list_elmt_on_off = element_list[element_list['SW_Module'] == 'S_ELMT_ON_OFF']
print(element_list_elmt_on_off.to_string())

                Prop_PID_Element Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                               
1.304                     [M, P]          Vacuum pump   S_ELMT_ON_OFF            1                                                                                                            
1.314                     [M, P]          Vacuum pump   S_ELMT_ON_OFF            2                                                                                                            
1.324                     [M, P]          Vacuum pump   S_ELMT_ON_OFF            3                                                                                                            
1.334                     [M, P]          Vac

In [787]:
element_list_limit_switch = element_list[element_list['SW_Module'] == 'S_LIMIT_SWITCH']
print(element_list_limit_switch.to_string())

                Prop_PID_Element Tag_Number_Definition       SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                
1.308                     [FSA-]           Flow-Switch  S_LIMIT_SWITCH            1                                                                                                            
1.318                     [FSA-]           Flow-Switch  S_LIMIT_SWITCH            2                                                                                                            
1.328                     [FSA-]           Flow-Switch  S_LIMIT_SWITCH            3                                                                                                            
1.338                     [FSA-]        

In [788]:
element_list_limit_hi_lo = element_list[element_list['SW_Module'] == 'S_LIMIT_HI_LO']
print(element_list_limit_hi_lo.to_string())

                 Prop_PID_Element          Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                         
1.108                       [PDI]    Diferential pressure sensor  S_LIMIT_HI_LO            1                                                                                                            
1.158                       [PDI]    Diferential pressure sensor  S_LIMIT_HI_LO            2                                                                                                            
1.208                       [PDI]   Diferential pressure sensor   S_LIMIT_HI_LO            3                                                                                                        

In [789]:
element_list_leak = element_list[element_list['SW_Module'] == 'S_LEAK']
print(element_list_leak.to_string())

                Prop_PID_Element Tag_Number_Definition SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                          
1.825              [FIA+, H, YC]      Flow rate sensor    S_LEAK            1                                                                                                            


In [790]:
#Particle counter modbus tcp
element_list_pms_modbus_tcp = element_list[element_list['SW_Module'] == 'S_PMS_MODBUS_TCP']
print(element_list_pms_modbus_tcp.to_string())


                Prop_PID_Element Tag_Number_Definition         SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                                  
1.30             [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            1                                                                                                            
1.31             [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            2                                                                                                            
1.32             [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            3                                                                                                            
1.33             [FA-, QIA+, X

In [791]:
#Air sampler modbus
element_list_aas_mas_tcp = element_list[element_list['SW_Module'] == 'S_AAS_MAS_TCP']
print(element_list_aas_mas_tcp.to_string())

                Prop_PID_Element Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
Prop_PID_Number                                                                                                                                                                               
1.35                     [P, XA]           Air sampler  S_AAS_MAS_TCP            1                                                                                                            
1.36                     [P, XA]           Air sampler  S_AAS_MAS_TCP            2                                                                                                            
1.37                     [P, XA]           Air sampler  S_AAS_MAS_TCP            3                                                                                                            
2.35                     [P, XA]     "2.35" n

In [792]:
#element_list_motors = element_list[element_list['SW_Module'] == 'S_MOTOR1']
#print(element_list_motors.to_string())

In [793]:
#Group by SW_Module
#el_sortby_sw_module = element_list.groupby(['SW_Module']).agg(list).copy()
#print(el_sortby_sw_module.to_string())

In [794]:
#Group by pdi number again
#this time
#Group by PID number
#element_list = element_list.groupby(['Prop_PID_Number']).agg(list).copy()

In [795]:
#print(element_list.to_string())

In [796]:
#list_1100 = element_list.loc[1.100].values.tolist()
#list_1100[0].sort()

In [797]:
#print(list_1100[0])

end_new code

In [798]:
#Remove duplicates - WARNING: It removes duplicates alarms as 1.310 Particle counter 0.5 um and 5.0um (maybe add exception when is an PCounter and duplicates elements)

#df_data_excel_uniques = df_data_excel.drop_duplicates()

In [799]:
#df_data_excel_uniques.count()

In [800]:
#gkk = df_data_excel_uniques.groupby(['Prop_PID_Number', 'Prop_PID_Element'])
#gkk.first()

In [801]:
#Get uniques values from the PID Elements (the columns for the matrix)
#df_unique_elememts = df_data_excel.Prop_PID_Element.unique()
#df_unique_elememts.sort()
#print(df_unique_elememts)




#Export Unique elements to excel file - Once


In [802]:
#Create a dataframe containing the elements
#pd_automation_code = pd.DataFrame(df_unique_elememts)
#Create the writer object
#automation_code = pd.ExcelWriter('AutomationCode.xlsx')
#Write dataframe to excel file
#pd_automation_code.to_excel(automation_code)
#Save excel 
#automation_code.save()
#Close excel
#automation_code.close()


In [803]:
#Sort by PID number
#df_data_excel_sorted = df_data_excel_uniques.sort_values(by='Prop_PID_Number')

In [804]:
#Set PID number as multi-index (group by does better)
#df_data_excel_sorted.set_index(['Prop_PID_Number','Prop_PID_Element'])

In [805]:
#df_data_excel_sorted.head(1000)

In [806]:
#Add column, load data from Automation Code and mix
#df_data_excel_sorted['Automation Code'] = ''


In [807]:
#df_data_excel_sorted

In [808]:
#Load Automation Code excel file
#df_automation_code = pd.read_excel('AutomationCode.xlsx')
#df_automation_code.head()

In [809]:
#Lookup
#df = df_data_excel
#df['Automation_Code'] = ''
#print (df)

In [810]:
# Deprecated df['Automation Code'] = df.set_index('Prop_PID_Element').lookup(df_automation_code.Prop_PID_Element, df_automation_code.Automation_Code)

In [811]:
#pd.melt(df,)
#print(df.combine_first(df_automation_code))

In [812]:
#documents = [['SW_MODULE', 'SW_MODULE_NR'], ['CONN1', 'CONN1_NR'], ['CONN2', 'CONN2_NR'], ['CONN3', 'CONN3_NR'], 'AM', 'HW']
#documents = ['EL', 'AM', 'HW', 'FN']

In [813]:
#
#df[df_unique_elememts] = pd.NaT
#documents = ['EL', 'AM', 'HW', 'FN']
#df[documents] = pd.NaT
#print(df.to_string())

Element list